In [ ]:
# Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

# 필요 라이브러리 설치
!pip install pyheif Pillow

import os
import pyheif
from PIL import Image

# 변환 함수 정의
def convert_images_to_png(input_folder, output_folder):
    """
    Google Drive 폴더 안의 heic와 jpg 파일을 png로 변환하는 함수
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, dirs, files in os.walk(input_folder):
        for file in files:
            input_path = os.path.join(root, file)
            output_path = os.path.join(output_folder, os.path.splitext(file)[0] + ".png")

            try:
                if file.lower().endswith(".heic"):
                    heif_file = pyheif.read(input_path)
                    image = Image.frombytes(
                        heif_file.mode,
                        heif_file.size,
                        heif_file.data,
                        "raw",
                        heif_file.mode,
                        heif_file.stride,
                    )
                    image.save(output_path, "PNG")
                    print(f"Converted HEIC: {file} -> {output_path}")
                elif file.lower().endswith(".jpg"):
                    image = Image.open(input_path)
                    image.save(output_path, "PNG")
                    print(f"Converted JPG: {file} -> {output_path}")
            except Exception as e:
                print(f"Failed to convert {file}: {e}")

# Google Drive에서 입력 및 출력 폴더 경로 설정
input_folder = '/content/drive/MyDrive/ATM/가공전'  # 변환할 파일이 있는 폴더 경로
output_folder = '/content/drive/MyDrive/ATM/png'  # 변환된 파일을 저장할 폴더 경로

# 변환 실행
convert_images_to_png(input_folder, output_folder)


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 33.1 MB/s eta 0:00:00
Converted HEIC: 20241118_160047.heic -> /content/drive/MyDrive/ATM/png/20241118_160047.png
Converted HEIC: 20241118_160042.heic -> /content/drive/MyDrive/ATM/png/20241118_160042.png
Converted HEIC: 20241118_160040(0).heic -> /content/drive/MyDrive/ATM/png/20241118_160040(0).png
Converted HEIC: 20241118_160040.heic -> /content/drive/MyDrive/ATM/png/20241118_160040.png
Converted HEIC: 20241118_160039(0).heic -> /content/drive/MyDrive/ATM/png/20241118_160039(0).png
Converted HEIC: 20241118_160039.heic -> /content/drive/MyDrive/ATM/png/20241118_160039.png
Converted HEIC: 20241118_160038(0).heic -> /content/drive/MyDrive/ATM/png/20241118_160038(0).png
Converted HEIC: 20241118_160038.heic -> /content/drive/MyDrive/ATM/png/20241118_160038.png
Converted HEIC: 20241118_160037(0).heic -> /content/drive/MyDrive/ATM/png/20241118_160037(0).png
Converted HEIC: 20241118_160037.heic -> /content/drive

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 라이브러리 설치
!pip install torch torchvision pillow numpy

import os
import numpy as np
from PIL import Image
import torchvision.transforms as T
import random

# 입력 폴더와 출력 폴더 설정
input_folder = '/content/drive/MyDrive/ATM/png'  # 원본 이미지 폴더
output_folder = '/content/drive/MyDrive/ATM/가공'  # 변환된 이미지 저장 폴더

# 출력 폴더 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 랜덤 노이즈 추가 함수
def add_random_noise(image):
    """이미지에 랜덤 노이즈 추가"""
    np_image = np.array(image)  # 이미지를 NumPy 배열로 변환
    noise = np.random.normal(loc=0, scale=random.randint(10, 50), size=np_image.shape)  # 노이즈 생성 (표준편차 랜덤)
    noisy_image = np_image + noise  # 원본 이미지에 노이즈 추가
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)  # 값 제한 및 정수형 변환
    return Image.fromarray(noisy_image)  # PIL 이미지로 변환

# 랜덤 증강 변환 정의 함수
def random_transformations():
    """랜덤 증강 변환 정의"""
    transforms_list = []

    # 각 변환에 확률적으로 추가
    if random.random() < 0.5:
        transforms_list.append(T.RandomHorizontalFlip(p=1.0))  # 수평 뒤집기
    if random.random() < 0.5:
        transforms_list.append(T.RandomVerticalFlip(p=1.0))  # 수직 뒤집기
    if random.random() < 0.3:
        transforms_list.append(T.RandomGrayscale(p=1.0))  # 회색조 변환
    if random.random() < 0.7:
        transforms_list.append(T.ColorJitter(
            saturation=random.uniform(0.1, 1.0),
            brightness=random.uniform(0.1, 1.0)
        ))  # 랜덤 채도 및 밝기
    if random.random() < 0.5:
        transforms_list.append(T.RandomRotation(degrees=(-90, 90)))  # 범위로 설정

    # 변환 리스트를 Compose로 묶기
    return T.Compose(transforms_list)

# 이미지 증강 및 저장 함수
def augment_and_save_images(input_folder, output_folder):
    """이미지 증강 및 저장"""
    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.png'):  # PNG 파일만 처리
            input_path = os.path.join(input_folder, filename)

            # 원본 이미지를 저장
            image = Image.open(input_path)
            base_name = os.path.splitext(filename)[0]
            original_output_path = os.path.join(output_folder, f"{base_name}_original.png")
            image.save(original_output_path)
            print(f"Saved original image: {original_output_path}")

            # 증강된 이미지 저장
            for i in range(5):  # 각 사진마다 5개의 증강 이미지 생성
                output_path = os.path.join(output_folder, f"{base_name}_augmented_{i + 1}.png")

                # 랜덤 변환 적용
                transformations = random_transformations()
                augmented_image = transformations(image)

                # 확률적으로 노이즈 추가 (30% 확률)
                if random.random() < 0.3:
                    augmented_image = add_random_noise(augmented_image)

                # 변환된 이미지 저장
                augmented_image.save(output_path)
                print(f"Processed and saved: {output_path}")

# 이미지 증강 수행
augment_and_save_images(input_folder, output_folder)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved original image: /content/drive/MyDrive/ATM/가공/20241118_160047_original.png
Processed and saved: /content/drive/MyDrive/ATM/가공/20241118_160047_augmented_1.png
Processed and saved: /content/drive/MyDrive/ATM/가공/20241118_160047_augmented_2.png
Processed and saved: /content/drive/MyDrive/ATM/가공/20241118_160047_augmented_3.png
Processed and saved: /content/drive/MyDrive/ATM/가공/20241118_160047_augmented_4.png
Processed and saved: /content/drive/MyDrive/ATM/가공/20241118_160047_augmented_5.png
Saved original image: /content/drive/MyDrive/ATM/가공/20241118_160042_original.png
Processed and saved: /content/drive/MyDrive/ATM/가공/20241118_160042_augmented_1.png
Processed and saved: /content/drive/MyDrive/ATM/가공/20241118_160042_augmented_2.png
Processed and saved: /content/drive/MyDrive/ATM/가공/20241118_160042_augmented_3.png
Processed and saved: /content/drive/MyDrive/A